# Transportation Simplex Algorithm

## Authors:
## Piotr Franc 160306
## Paweł Charkiewicz 160288

In [1]:
import numpy as np
import pulp
import time
from statistics import mean
from random import randint, random

The TSA implementation. Uses a single expanded problem matrix, which can accept any compination of intervals and constant values.

In [2]:
def TSA(problem_matrix: np.ndarray[np.float32]):
    iterations = 0
    chain_sizes = []

    # === INIT ===
    # 1. Initial tableau and intervals
    links_matrix = problem_matrix[2:, 2:]
    sources_intervals = problem_matrix[2:, :2]
    sinks_intervals = problem_matrix[:2, 2:].T

    # 3a. Deal with infinities
    low_supply_bounds = sources_intervals[:, 0]
    high_supply_bounds = sources_intervals[:, 1]

    low_demand_bounds = sinks_intervals[:, 0]
    high_demand_bounds = sinks_intervals[:, 1]

    min_total_supply = np.sum(low_supply_bounds)
    max_total_supply = np.sum(high_supply_bounds)

    min_total_demand = np.sum(low_demand_bounds)
    max_total_demand = np.sum(high_demand_bounds)

    if max_total_supply < min_total_demand or min_total_supply > max_total_demand:
        return -1, None

    if (max_total_supply == np.inf) ^ (max_total_demand == np.inf):
        if max_total_supply == np.inf:
            sources_intervals[sources_intervals == np.inf] = max_total_demand
        else:
            sinks_intervals[sinks_intervals == np.inf] = max_total_supply

    elif max_total_supply == np.inf and max_total_demand == np.inf:
        supply_bounds_unknown = np.isinf(high_supply_bounds)
        demand_bounds_unknown = np.isinf(high_demand_bounds)

        max_finite_supply = high_supply_bounds[~supply_bounds_unknown].sum()
        max_finite_demand = high_demand_bounds[~demand_bounds_unknown].sum()

        excess_supply_needed = max(0.0, min_total_demand - max_finite_supply)
        excess_demand_needed = max(0.0, min_total_supply - max_finite_demand)

        high_supply_bounds[supply_bounds_unknown] = low_supply_bounds[supply_bounds_unknown]
        high_demand_bounds[demand_bounds_unknown] = low_demand_bounds[demand_bounds_unknown]

        high_supply_bounds[np.flatnonzero(supply_bounds_unknown)] += excess_supply_needed
        high_demand_bounds[np.flatnonzero(demand_bounds_unknown)] += excess_demand_needed

    # 3b. Deal with big M
    max_link_cost = np.max(links_matrix[links_matrix != np.inf])
    big_M = max_link_cost * 100
    links_matrix[links_matrix == np.inf] = big_M

    # 4. Split
    if np.any(np.concatenate([
        high_supply_bounds < 0,
        high_demand_bounds < 0,
        low_supply_bounds < 0,
        low_demand_bounds < 0,
        high_supply_bounds < low_supply_bounds,
        high_demand_bounds < low_demand_bounds
    ])):
        return -1, None

    high_supply_bounds -= low_supply_bounds
    high_demand_bounds -= low_demand_bounds

    guaranteed_supply = np.flatnonzero(low_supply_bounds)
    guaranteed_demand = np.flatnonzero(low_demand_bounds)

    excess_supply = np.flatnonzero(high_supply_bounds)
    excess_demand = np.flatnonzero(high_demand_bounds)

    split_supply_indices = np.concatenate([guaranteed_supply, excess_supply])
    split_demand_indices = np.concatenate([guaranteed_demand, excess_demand])

    work_array = np.zeros(
        (guaranteed_supply.size + excess_supply.size,
         guaranteed_demand.size + excess_demand.size, 3),
        dtype=np.float32
    )

    supply_mesh, demand_mesh = np.meshgrid(split_supply_indices,
                                           split_demand_indices,
                                           indexing='ij')

    work_array[:, :, 0] = links_matrix[supply_mesh, demand_mesh]

    # 5 Add dummies
    split_guaranteed_supply = low_supply_bounds[guaranteed_supply]
    split_excess_supply = high_supply_bounds[excess_supply]

    split_guaranteed_demand = low_demand_bounds[guaranteed_demand]
    split_excess_demand = high_demand_bounds[excess_demand]

    split_supply = np.concatenate([split_guaranteed_supply, split_excess_supply])
    split_demand = np.concatenate([split_guaranteed_demand, split_excess_demand])

    if split_excess_supply.sum() > 0:
        work_array = np.pad(work_array, ((0, 0), (0, 1), (0, 0)),
                            mode='constant', constant_values=0)

        work_array[guaranteed_supply.size:, -1, 0] = 0
        work_array[:guaranteed_supply.size, -1, 0] = big_M

        split_demand = np.append(split_demand, split_excess_demand.sum())

    if split_excess_demand.sum() > 0:
        work_array = np.pad(work_array, ((0, 1), (0, 0), (0, 0)),
                            mode='constant', constant_values=0)

        work_array[-1, guaranteed_demand.size:, 0] = 0
        work_array[-1, :guaranteed_demand.size, 0] = big_M

        split_supply = np.append(split_supply, split_excess_supply.sum())

    # 6. Balance
    total_split_supply = split_supply.sum()
    total_split_demand = split_demand.sum()

    if total_split_supply > total_split_demand:
        split_demand[-1] += total_split_supply - total_split_demand

    if total_split_demand > total_split_supply:
        split_supply[-1] += total_split_demand - total_split_supply

    problem_size = work_array.shape[0] * work_array.shape[1]

    # 7. Construct workable tableau... DONE!

    # 8. Construct baseline solution with NW corner rule
    supply_left = split_supply.copy()
    demand_left = split_demand.copy()
    for i in range(work_array.shape[0]):
        for j in range(work_array.shape[1]):
            if supply_left[i] == 0 or demand_left[j] == 0:
                continue

            allocation = np.minimum(supply_left[i], demand_left[j])
            work_array[i, j, 1] = allocation
            work_array[i, j, 2] = 1
            supply_left[i] -= allocation
            demand_left[j] -= allocation

    # 9. Compute initial u, v
    u_values = np.full(work_array.shape[0], np.nan, dtype=np.float32)
    v_values = np.full(work_array.shape[1], np.nan, dtype=np.float32)

    u_values[np.argmax(np.sum(work_array[:, :, 2], axis=1))] = 0

    u_computed = np.isfinite(u_values)
    v_computed = np.isfinite(v_values)

    while not (np.all(u_computed) and np.all(v_computed)):
        progress = False
        for i in range(work_array.shape[0]):
            for j in range(work_array.shape[1]):
                if work_array[i, j, 2] == 1:
                    if u_computed[i] and not v_computed[j]:
                        v_values[j] = work_array[i, j, 0] - u_values[i]
                        v_computed[j] = True
                        progress = True
                    elif v_computed[j] and not u_computed[i]:
                        u_values[i] = work_array[i, j, 0] - v_values[j]
                        u_computed[i] = True
                        progress = True
        if not progress:
            break

    if np.logical_or(~np.all(u_computed), ~np.all(v_computed)):
        return -1, None

    # 10. Compute reduced costs
    for i in range(work_array.shape[0]):
        for j in range(work_array.shape[1]):
            if work_array[i, j, 2] == 0:
                work_array[i, j, 1] = work_array[i, j, 0] - (u_values[i] + v_values[j])

    # === LOOP ===
    while np.any(work_array[:, :, 1] < 0):
        iterations += 1

        # 11. Pick node for swap
        nonbasic_mask = work_array[:, :, 2] == 0
        masked_reduced_costs = work_array[:, :, 1].copy()
        masked_reduced_costs[~nonbasic_mask] = np.inf

        target_row_index, target_col_index = np.unravel_index(
            np.argmin(masked_reduced_costs), masked_reduced_costs.shape
        )

        if masked_reduced_costs[target_row_index, target_col_index] >= 0:
            break

        work_array[target_row_index, target_col_index, 1] = 0.0

        # 12. Chain rule (find cycle)
        basic_mask = work_array[:, :, 2] == 1
        num_rows, num_cols = basic_mask.shape

        basic_columns_in_row = [
            np.flatnonzero(basic_mask[row_index]).tolist()
            for row_index in range(num_rows)
        ]
        basic_rows_in_column = [
            np.flatnonzero(basic_mask[:, col_index]).tolist()
            for col_index in range(num_cols)
        ]

        ROW_NODE, COLUMN_NODE = 0, 1

        start_node = (ROW_NODE, target_row_index)
        goal_node = (COLUMN_NODE, target_col_index)

        node_stack = [start_node]
        predecessor = {start_node: None}

        while node_stack:
            current_node_type, current_node_index = node_stack.pop()

            if (current_node_type, current_node_index) == goal_node:
                break

            if current_node_type == ROW_NODE:
                for column_index in basic_columns_in_row[current_node_index]:
                    neighbor_node = (COLUMN_NODE, column_index)
                    if neighbor_node not in predecessor:
                        predecessor[neighbor_node] = (current_node_type, current_node_index)
                        node_stack.append(neighbor_node)
            else:
                for row_index in basic_rows_in_column[current_node_index]:
                    neighbor_node = (ROW_NODE, row_index)
                    if neighbor_node not in predecessor:
                        predecessor[neighbor_node] = (current_node_type, current_node_index)
                        node_stack.append(neighbor_node)

        if goal_node not in predecessor:
            return -1, None

        node_path = []
        current_node = goal_node
        while current_node is not None:
            node_path.append(current_node)
            current_node = predecessor[current_node]
        node_path.reverse()

        cycle = [(target_row_index, target_col_index)]
        for (node_type_a, index_a), (node_type_b, index_b) in zip(node_path[:-1], node_path[1:]):
            if node_type_a == ROW_NODE and node_type_b == COLUMN_NODE:
                row_index, column_index = index_a, index_b
            elif node_type_a == COLUMN_NODE and node_type_b == ROW_NODE:
                row_index, column_index = index_b, index_a
            else:
                return -1, None
            cycle.append((row_index, column_index))

        # Track chain size
        chain_sizes.append(len(cycle))

        minus_cycle_indices = np.arange(1, len(cycle), 2)
        minus_allocations = np.array([
            work_array[cycle[i][0], cycle[i][1], 1] for i in minus_cycle_indices
        ])

        leaving_cycle_index = int(minus_cycle_indices[np.argmin(minus_allocations)])
        leaving_row_index, leaving_col_index = cycle[leaving_cycle_index]

        # 13. Update tableau
        donors = cycle[1::2]
        recipients = cycle[0::2]

        theta = min(work_array[row_index, col_index, 1] for row_index, col_index in donors)

        work_array[leaving_row_index, leaving_col_index, 2] = 0
        work_array[target_row_index, target_col_index, 2] = 1

        for row_index, col_index in recipients:
            work_array[row_index, col_index, 1] += theta

        for row_index, col_index in donors:
            work_array[row_index, col_index, 1] -= theta

        # 14. Recompute u and v, then reduced costs
        u_values[:] = np.nan
        v_values[:] = np.nan
        u_values[np.argmax(np.sum(work_array[:, :, 2], axis=1))] = 0

        u_computed = np.isfinite(u_values)
        v_computed = np.isfinite(v_values)

        while not (np.all(u_computed) and np.all(v_computed)):
            progress = False
            for i in range(work_array.shape[0]):
                for j in range(work_array.shape[1]):
                    if work_array[i, j, 2] == 1:
                        if u_computed[i] and not v_computed[j]:
                            v_values[j] = work_array[i, j, 0] - u_values[i]
                            v_computed[j] = True
                            progress = True
                        elif v_computed[j] and not u_computed[i]:
                            u_values[i] = work_array[i, j, 0] - v_values[j]
                            u_computed[i] = True
                            progress = True
            if not progress:
                break

        if np.logical_or(~np.all(u_computed), ~np.all(v_computed)):
            return -1, None

        for i in range(work_array.shape[0]):
            for j in range(work_array.shape[1]):
                if work_array[i, j, 2] == 0:
                    work_array[i, j, 1] = work_array[i, j, 0] - (u_values[i] + v_values[j])

    # === LOOP END ===
    cost = np.sum(work_array[:, :, 0] * work_array[:, :, 1] * work_array[:, :, 2])

    metrics = {
        'iterations': iterations,
        'chain_sizes': chain_sizes,
        'avg_chain_size': mean(chain_sizes) if chain_sizes else 0,
        'max_chain_size': max(chain_sizes) if chain_sizes else 0,
        'min_chain_size': min(chain_sizes) if chain_sizes else 0,
        'problem_size': problem_size
    }

    if np.any((work_array[..., 2] == 1) & (work_array[..., 0] == big_M)):
        return -1, None

    return cost, metrics


Small transportation problem solver based on PuLP. Uses constant capacity values.

In [3]:
def PuTPS(problem_matrix):
    N = problem_matrix.shape[0] - 2

    s = problem_matrix[2:, 0]
    d = problem_matrix[0, 2:]
    c_matrix = problem_matrix[2:, 2:]

    c_matrix[np.isinf(c_matrix)] = 10 ** 6

    model = pulp.LpProblem("PuLP_TP_solver", pulp.LpMinimize)
    cost = pulp.LpVariable.dicts("cost", (range(N), range(N)), lowBound=0)

    model += pulp.lpSum(c_matrix[i, j] * cost[i][j] for i in range(N) for j in range(N))

    for i in range(N):
        model += pulp.lpSum(cost[i][j] for j in range(N)) == s[i]
        model += pulp.lpSum(cost[j][i] for j in range(N)) == d[i]

    model.solve(pulp.PULP_CBC_CMD(msg=False))
    return model.objective.value()

TS problem generator of size N. Uses constant capacity values instead of intervals for PuLP compatibility.

In [4]:
ZERO_ROW_CHANCE = 0.05
ZERO_COL_CHANCE = 0.05
M_CHANCE = 0.1

def ts_problem(N):
    s = np.random.randint(0, 131, N)

    while True:
        d = np.random.randint(0, 101, N)
        supply_left = s.sum() - d[:-1].sum()
        if supply_left < 0:
            continue
        d[-1] = supply_left
        break

    link_costs = np.random.randint(0, 31, (N, N)).astype(np.float32)
    m_mask = np.random.rand(N, N) < M_CHANCE
    link_costs[m_mask] = np.inf

    if random() < ZERO_ROW_CHANCE:
        link_costs[randint(0, N - 1), :] = 0
    if random() < ZERO_COL_CHANCE:
        link_costs[:, randint(0, N - 1)] = 0

    supplies = s.astype(np.float32)
    demands = d.astype(np.float32)

    out = np.zeros((N + 2, N + 2), dtype=np.float32)
    out[2:, 2:] = link_costs
    out[2:, 0] = supplies
    out[2:, 1] = supplies
    out[0, 2:] = demands
    out[1, 2:] = demands

    return out

Test script. We verify the performance on four problem sizes. Collecting extra metadata adds significant performance overhead for the TSA. The solution quality is equal to the PuLP based solver.

In [5]:
problem_sizes = [5, 10, 15, 30]
num_runs = 100

print("=" * 100)
print("TRANSPORTATION SIMPLEX ALGORITHM - PERFORMANCE ANALYSIS")
print("=" * 100)
print(f"\nProblem sizes: {problem_sizes}")
print(f"Runs per size: {num_runs}\n")

for N in problem_sizes:
    print(f"Processing N={N}...", end='', flush=True)

    tsa_costs = []
    pulp_costs = []
    tsa_times = []
    pulp_times = []
    iterations_list = []
    avg_chain_sizes = []
    max_chain_sizes = []
    min_chain_sizes = []

    successful_cases = 0
    invalid_problems = 0

    for case in range(num_runs):
        problem = ts_problem(N)

        start = time.time()
        tsa_result = TSA(problem)

        if isinstance(tsa_result, tuple):
            tsa_cost, metrics = tsa_result
        else:
            tsa_cost = tsa_result
            metrics = None

        if tsa_cost == -1:
            invalid_problems += 1
            continue

        tsa_times.append(time.time() - start)
        tsa_costs.append(tsa_cost)

        if metrics:
            iterations_list.append(metrics['iterations'])
            avg_chain_sizes.append(metrics['avg_chain_size'])
            max_chain_sizes.append(metrics['max_chain_size'])
            min_chain_sizes.append(metrics['min_chain_size'])

        start = time.time()
        pulp_cost = PuTPS(problem)
        pulp_times.append(time.time() - start)
        pulp_costs.append(pulp_cost)

        successful_cases += 1

    print(f" Done ({successful_cases} successful, {invalid_problems} invalid)")

    tsa_costs = np.array(tsa_costs)
    pulp_costs = np.array(pulp_costs)
    tsa_times = np.array(tsa_times)
    pulp_times = np.array(pulp_times)
    iterations_list = np.array(iterations_list)
    avg_chain_sizes = np.array(avg_chain_sizes)
    max_chain_sizes = np.array(max_chain_sizes)
    min_chain_sizes = np.array(min_chain_sizes)

    # Results for size N
    print(f"\n{'=' * 100}")
    print(f"RESULTS FOR N={N} ({successful_cases} successful, {invalid_problems} invalid)")
    print(f"{'=' * 100}\n")

    # Check for misformed problems
    print(f"{'PROBLEM FEASIBILITY':<30} {'Count':<20} {'Percentage':<20}")
    print("-" * 70)
    print(f"{'Valid problems':<30} {successful_cases:<20} {successful_cases/num_runs*100:<20.2f}%")
    print(f"{'Invalid/infeasible':<30} {invalid_problems:<20} {invalid_problems/num_runs*100:<20.2f}%")

    if successful_cases == 0:
        print("\nNo successful cases to analyze.\n")
        continue

    # Compare costs, should be one and the same
    print(f"\n{'COST ANALYSIS':<30} {'TSA':<20} {'PuLP':<20} {'Difference':<20}")
    print("-" * 90)
    print(
        f"{'Average Cost':<30} {np.mean(tsa_costs):<20.2f} {np.mean(pulp_costs):<20.2f} {np.abs(np.mean(tsa_costs) - np.mean(pulp_costs)):<20.2f}")
    print(
        f"{'Std Dev':<30} {np.std(tsa_costs):<20.2f} {np.std(pulp_costs):<20.2f} {np.abs(np.std(tsa_costs) - np.std(pulp_costs)):<20.2f}")
    print(
        f"{'Min Cost':<30} {np.min(tsa_costs):<20.2f} {np.min(pulp_costs):<20.2f} {np.abs(np.min(tsa_costs) - np.min(pulp_costs)):<20.2f}")
    print(
        f"{'Max Cost':<30} {np.max(tsa_costs):<20.2f} {np.max(pulp_costs):<20.2f} {np.abs(np.max(tsa_costs) - np.max(pulp_costs)):<20.2f}")

    # Algorithm performance metrics
    print(f"\n{'ALGORITHM PERFORMANCE':<30} {'Mean':<15} {'Std':<15} {'Min':<15} {'Max':<15}")
    print("-" * 90)
    print(
        f"{'Iterations to optimum':<30} {np.mean(iterations_list):<15.2f} {np.std(iterations_list):<15.2f} {np.min(iterations_list):<15.0f} {np.max(iterations_list):<15.0f}")
    print(
        f"{'Average chain size':<30} {np.mean(avg_chain_sizes):<15.2f} {np.std(avg_chain_sizes):<15.2f} {np.min(avg_chain_sizes):<15.2f} {np.max(avg_chain_sizes):<15.2f}")
    print(
        f"{'Maximum chain size':<30} {np.mean(max_chain_sizes):<15.2f} {np.std(max_chain_sizes):<15.2f} {np.min(max_chain_sizes):<15.0f} {np.max(max_chain_sizes):<15.0f}")
    print(
        f"{'Minimum chain size':<30} {np.mean(min_chain_sizes):<15.2f} {np.std(min_chain_sizes):<15.2f} {np.min(min_chain_sizes):<15.0f} {np.max(min_chain_sizes):<15.0f}")

    # Compare execution time
    tsa_avg_ms = np.mean(tsa_times) * 1000
    pulp_avg_ms = np.mean(pulp_times) * 1000
    speedup = pulp_avg_ms / tsa_avg_ms if tsa_avg_ms > 0 else 0

    print(f"\n{'TIME PERFORMANCE (ms)':<30} {'TSA':<20} {'PuLP':<20} {'Speedup':<20}")
    print("-" * 90)
    print(f"{'Average Time':<30} {tsa_avg_ms:<20.4f} {pulp_avg_ms:<20.4f} {speedup:<20.2f}x")
    print(f"{'Total Time':<30} {np.sum(tsa_times) * 1000:<20.2f} {np.sum(pulp_times) * 1000:<20.2f}")

    # Wins/losses/ties, should be one and the same
    cost_diff = tsa_costs - pulp_costs
    tsa_wins = np.sum(cost_diff < -0.01)
    pulp_wins = np.sum(cost_diff > 0.01)
    ties = np.sum(np.abs(cost_diff) <= 0.01)

    print(f"\n{'SOLUTION QUALITY':<30} {'Count':<20} {'Percentage':<20}")
    print("-" * 70)
    print(f"{'TSA Better':<30} {tsa_wins:<20} {tsa_wins / successful_cases * 100:<20.2f}%")
    print(f"{'PuLP Better':<30} {pulp_wins:<20} {pulp_wins / successful_cases * 100:<20.2f}%")
    print(f"{'Equivalent (±0.01)':<30} {ties:<20} {ties / successful_cases * 100:<20.2f}%")
    print()

print("=" * 100)
print("ANALYSIS COMPLETE")
print("=" * 100)

TRANSPORTATION SIMPLEX ALGORITHM - PERFORMANCE ANALYSIS

Problem sizes: [5, 10, 15, 30]
Runs per size: 100

Processing N=5... Done (91 successful, 9 invalid)

RESULTS FOR N=5 (91 successful, 9 invalid)

PROBLEM FEASIBILITY            Count                Percentage          
----------------------------------------------------------------------
Valid problems                 91                   91.00               %
Invalid/infeasible             9                    9.00                %

COST ANALYSIS                  TSA                  PuLP                 Difference          
------------------------------------------------------------------------------------------
Average Cost                   3332.82              3332.82              0.00                
Std Dev                        1513.45              1513.45              0.00                
Min Cost                       305.00               305.00               0.00                
Max Cost                       6913.0